# Data Science Final Project
by: Christopher Yarbro and Lakeland Galinson

## Intro
Our names are Christopher Yarbro and Lakeland Galinson. We are seniors at Tulane University studying Computer Science. This website will host the code, figures, and anything else necessary for our Final Project in Data Science

## Organization
Our first step was to create a git repository hosted on github that we can both access and edit. We are roommates, so we have been able to meet regularly to discuss our plans for the project.

## Data Sets

### New Orleans Electronic Police Reports 2025
This dataset is a collection of police reports from 2025 in New Orleans. It contains information about location, date, type of offense, imformation about the offender and victim(s).

The dataset can be found [here](https://data.nola.gov/Public-Safety-and-Preparedness/Electronic-Police-Report-2025/agqi-9adb/about_data)

We will load, clean, and explore this dataset below.